# Import Libraries

In [18]:
import json
import pandas as pd
import pandas_market_calendars as mcal
from sqlalchemy import create_engine, text, inspect

# Connect to Database

In [14]:
with open("../config/credentials.json") as f:
    creds = json.load(f)

username = creds["POSTGRES_USERNAME"]
password = creds["POSTGRES_PASSWORD"]
host = creds["POSTGRES_HOST"]
port = creds["POSTGRES_PORT"]
dbname = creds["POSTGRES_DBNAME"]

DB_URL = f"postgresql://{username}:{password}@{host}:{port}/{dbname}"
engine = create_engine(DB_URL)

# Validate Ticker Metadata

In [15]:
with engine.connect() as conn:
    # Get total row count
    total_rows = conn.execute(text("SELECT COUNT(*) FROM quant.ticker_metadata")).scalar()

    # Check for rows with NULLs (excluding primary key)
    nulls = conn.execute(text("""
        SELECT *
        FROM quant.ticker_metadata
        WHERE "StartDate" IS NULL OR "EndDate" IS NULL OR "SectorETF" IS NULL;
    """)).fetchall()

    # Check for duplicate Ticker values
    duplicates = conn.execute(text("""
        SELECT "Ticker", COUNT(*) as count
        FROM quant.ticker_metadata
        GROUP BY "Ticker"
        HAVING COUNT(*) > 1;
    """)).fetchall()

# Display results
print(f"Total rows in quant.ticker_metadata: {total_rows}")

print("\nNULL Entries in Non-Key Columns:")
if nulls:
    for row in nulls:
        print(row)
else:
    print("No NULL values found.")

print("\nDuplicate Ticker Entries:")
if duplicates:
    for row in duplicates:
        print(f"Ticker: {row[0]}, Count: {row[1]}")
else:
    print("No duplicate tickers found.")


Total rows in quant.ticker_metadata: 648

NULL Entries in Non-Key Columns:
No NULL values found.

Duplicate Ticker Entries:
No duplicate tickers found.


# Validate Sector ETF Data

In [21]:
# Find NYSE trading days
nyse = mcal.get_calendar("NYSE")
schedule = nyse.schedule(start_date="2017-01-01", end_date="2025-06-22")
expected_dates = schedule.index.date
expected_date_set = set(expected_dates)

# Expected 11 Sector ETFs (standard GICS sectors)
expected_etfs = {
    "XLB", "XLC", "XLE", "XLF", "XLI", "XLK",
    "XLP", "XLRE", "XLU", "XLV", "XLY"
}

# Column names from schema
columns = [
    "etf", "date", "close", "high", "low", "open", "volume",
    "adjclose", "adjhigh", "adjlow", "adjopen", "adjvolume",
    "divcash", "splitfactor"
]

with engine.connect() as conn:
    # Row count
    total_rows = conn.execute(text("SELECT COUNT(*) FROM quant.sector_etf_prices")).scalar()

    # Null check
    null_query = f"""
        SELECT *
        FROM quant.sector_etf_prices
        WHERE {" OR ".join([f'"{col}" IS NULL' for col in columns])};
    """
    nulls = conn.execute(text(null_query)).fetchall()

    # Duplicate ETF-date entries
    duplicates = conn.execute(text("""
        SELECT "etf", "date", COUNT(*) as count
        FROM quant.sector_etf_prices
        GROUP BY "etf", "date"
        HAVING COUNT(*) > 1;
    """)).fetchall()

    # All ETF + Date rows
    rows = conn.execute(text("""
        SELECT "etf", "date"
        FROM quant.sector_etf_prices
    """)).fetchall()

# DataFrame to check missing dates
df = pd.DataFrame(rows, columns=["etf", "date"])
df["date"] = pd.to_datetime(df["date"]).dt.date

# Check for missing trading days per ETF
missing_days = {}
for etf in df["etf"].unique():
    actual_dates = set(df[df["etf"] == etf]["date"])
    missing = expected_date_set - actual_dates
    if missing:
        missing_days[etf] = sorted(missing)

# Check if all 11 ETFs are present
present_etfs = set(df["etf"].unique())
missing_etfs = expected_etfs - present_etfs
extra_etfs = present_etfs - expected_etfs

# Display Results
print(f"Total rows in quant.sector_etf_prices: {total_rows}")

print("\nNULL Entries in Any Column:")
if nulls:
    print(f"{len(nulls)} rows with NULLs. Sample:")
    for row in nulls[:5]:
        print(row)
else:
    print("No NULL values found in any column.")

print("\nDuplicate Entries (ETF + Date):")
if duplicates:
    for row in duplicates:
        print(f"ETF: {row[0]}, Date: {row[1]}, Count: {row[2]}")
else:
    print("No duplicate (ETF, Date) pairs found.")

print("\nMissing NYSE Trading Days per ETF:")
if missing_days:
    for etf, dates in missing_days.items():
        print(f"{etf}: {len(dates)} missing (e.g., {dates[:5]}...)")
else:
    print("All ETFs have full NYSE trading day coverage.")

print("\nETF Coverage Check:")
if missing_etfs:
    print(f"Missing ETFs: {missing_etfs}")
else:
    print("All 11 GICS sector ETFs are present.")

if extra_etfs:
    print(f"Extra unexpected ETFs found: {extra_etfs}")


Total rows in quant.sector_etf_prices: 23408

NULL Entries in Any Column:
No NULL values found in any column.

Duplicate Entries (ETF + Date):
No duplicate (ETF, Date) pairs found.

Missing NYSE Trading Days per ETF:
All ETFs have full NYSE trading day coverage.

ETF Coverage Check:
All 11 GICS sector ETFs are present.


# Validate Macro Indicators

In [23]:
# NYSE trading calendar: 2017-01-01 to 2025-06-22
nyse = mcal.get_calendar("NYSE")
schedule = nyse.schedule(start_date="2017-01-01", end_date="2025-06-22")
expected_dates = schedule.index.date
expected_date_set = set(expected_dates)

# Expected macro indicator tickers (update as needed)
expected_macros = {
    "DXY",
    "VIX",
    "10Y",
    "5Y",
    "3M",
    "OIL",
    "GOLD"
}

# Column names from schema
columns = ["ticker", "date", "close", "high", "low", "open", "volume"]

with engine.connect() as conn:
    # Total row count
    total_rows = conn.execute(text("SELECT COUNT(*) FROM quant.macro_indicators")).scalar()

    # Null check in all columns
    null_query = f"""
        SELECT *
        FROM quant.macro_indicators
        WHERE {" OR ".join([f'"{col}" IS NULL' for col in columns])};
    """
    nulls = conn.execute(text(null_query)).fetchall()

    # Check for duplicate (Ticker, Date) entries
    duplicates = conn.execute(text("""
        SELECT "ticker", "date", COUNT(*) as count
        FROM quant.macro_indicators
        GROUP BY "ticker", "date"
        HAVING COUNT(*) > 1;
    """)).fetchall()

    # Load all ticker/date rows
    rows = conn.execute(text("""
        SELECT "ticker", "date"
        FROM quant.macro_indicators
    """)).fetchall()

# DataFrame for missing date checks
df = pd.DataFrame(rows, columns=["ticker", "date"])
df["date"] = pd.to_datetime(df["date"]).dt.date

# Missing NYSE trading days check per macro
missing_days = {}
for ticker in df["ticker"].unique():
    actual_dates = set(df[df["ticker"] == ticker]["date"])
    missing = expected_date_set - actual_dates
    if missing:
        missing_days[ticker] = sorted(missing)

# Validate macro indicator coverage
present_macros = set(df["ticker"].unique())
missing_macros = expected_macros - present_macros
extra_macros = present_macros - expected_macros

# Display Results
print(f"Total rows in quant.macro_indicators: {total_rows}")

print("\nNULL Entries in Any Column:")
if nulls:
    print(f"{len(nulls)} rows with NULLs. Sample:")
    for row in nulls[:5]:
        print(row)
else:
    print("No NULL values found in any column.")

print("\nDuplicate Entries (Ticker + Date):")
if duplicates:
    for row in duplicates:
        print(f"Ticker: {row[0]}, Date: {row[1]}, Count: {row[2]}")
else:
    print("No duplicate (Ticker, Date) pairs found.")

print("\nMissing NYSE Trading Days per Macro Indicator:")
if missing_days:
    for ticker, dates in missing_days.items():
        print(f"{ticker}: {len(dates)} missing (e.g., {dates[:5]}...)")
else:
    print("All macros have full NYSE trading day coverage.")

print("\nMacro Ticker Coverage Check:")
if missing_macros:
    print(f"Missing macro indicators: {missing_macros}")
else:
    print("All 7 expected macro indicators are present.")

if extra_macros:
    print(f"Extra unexpected macro tickers found: {extra_macros}")


Total rows in quant.macro_indicators: 16658

NULL Entries in Any Column:
No NULL values found in any column.

Duplicate Entries (Ticker + Date):
No duplicate (Ticker, Date) pairs found.

Missing NYSE Trading Days per Macro Indicator:
GOLD: 1 missing (e.g., [datetime.date(2018, 1, 29)]...)

Macro Ticker Coverage Check:
All 7 expected macro indicators are present.


In [25]:
ticker = "GOLD"
missing_date = pd.to_datetime("2018-01-29").date()

with engine.connect() as conn:
    # Get count before
    before_count = conn.execute(text("""
        SELECT COUNT(*) FROM quant.macro_indicators
        WHERE ticker = :ticker
    """), {"ticker": ticker}).scalar()

    # Get the previous and next available trading days
    rows = conn.execute(text("""
        (
            SELECT * FROM quant.macro_indicators
            WHERE ticker = :ticker AND date < :missing_date
            ORDER BY date DESC LIMIT 1
        )
        UNION ALL
        (
            SELECT * FROM quant.macro_indicators
            WHERE ticker = :ticker AND date > :missing_date
            ORDER BY date ASC LIMIT 1
        )
    """), {"ticker": ticker, "missing_date": missing_date}).fetchall()

# Interpolate if both rows exist
if len(rows) == 2:
    df = pd.DataFrame(rows, columns=["ticker", "date", "close", "high", "low", "open", "volume"])
    df.set_index("date", inplace=True)
    df_sorted = df.sort_index()

    # Linear interpolation
    interp_row = df_sorted.reindex(
        pd.date_range(df_sorted.index.min(), df_sorted.index.max(), freq="D")
    ).interpolate(method='linear').loc[pd.Timestamp(missing_date)]

    # Insert interpolated row
    with engine.begin() as conn:
        conn.execute(text("""
            INSERT INTO quant.macro_indicators (
                ticker, date, close, high, low, open, volume
            ) VALUES (
                :ticker, :date, :close, :high, :low, :open, :volume
            )
        """), {
            "ticker": ticker,
            "date": missing_date,
            "close": float(interp_row["close"]),
            "high": float(interp_row["high"]),
            "low": float(interp_row["low"]),
            "open": float(interp_row["open"]),
            "volume": int(interp_row["volume"])
        })

    # Confirm count after insertion
    with engine.connect() as conn:
        after_count = conn.execute(text("""
            SELECT COUNT(*) FROM quant.macro_indicators
            WHERE ticker = :ticker
        """), {"ticker": ticker}).scalar()

    print(f"GOLD row count before: {before_count}")
    print(f"GOLD row count after:  {after_count}")
    print(f"Interpolated and inserted: {missing_date}")

else:
    print("Could not find surrounding trading days for interpolation.")


C:\Users\raira\AppData\Local\Temp\ipykernel_28708\805922407.py:35: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  ).interpolate(method='linear').loc[pd.Timestamp(missing_date)]


GOLD row count before: 2379
GOLD row count after:  2380
Interpolated and inserted: 2018-01-29


In [ ]:
# NYSE trading calendar: 2017-01-01 to 2025-06-22
nyse = mcal.get_calendar("NYSE")
schedule = nyse.schedule(start_date="2017-01-01", end_date="2025-06-22")
expected_dates = schedule.index.date
expected_date_set = set(expected_dates)

# Expected macro indicator tickers (update as needed)
expected_macros = {
    "DXY",
    "VIX",
    "10Y",
    "5Y",
    "3M",
    "OIL",
    "GOLD"
}

# Column names from schema
columns = ["ticker", "date", "close", "high", "low", "open", "volume"]

with engine.connect() as conn:
    # Total row count
    total_rows = conn.execute(text("SELECT COUNT(*) FROM quant.macro_indicators")).scalar()

    # Null check in all columns
    null_query = f"""
        SELECT *
        FROM quant.macro_indicators
        WHERE {" OR ".join([f'"{col}" IS NULL' for col in columns])};
    """
    nulls = conn.execute(text(null_query)).fetchall()

    # Check for duplicate (Ticker, Date) entries
    duplicates = conn.execute(text("""
        SELECT "ticker", "date", COUNT(*) as count
        FROM quant.macro_indicators
        GROUP BY "ticker", "date"
        HAVING COUNT(*) > 1;
    """)).fetchall()

    # Load all ticker/date rows
    rows = conn.execute(text("""
        SELECT "ticker", "date"
        FROM quant.macro_indicators
    """)).fetchall()

# DataFrame for missing date checks
df = pd.DataFrame(rows, columns=["ticker", "date"])
df["date"] = pd.to_datetime(df["date"]).dt.date

# Missing NYSE trading days check per macro
missing_days = {}
for ticker in df["ticker"].unique():
    actual_dates = set(df[df["ticker"] == ticker]["date"])
    missing = expected_date_set - actual_dates
    if missing:
        missing_days[ticker] = sorted(missing)

# Validate macro indicator coverage
present_macros = set(df["ticker"].unique())
missing_macros = expected_macros - present_macros
extra_macros = present_macros - expected_macros

# Display Results
print(f"Total rows in quant.macro_indicators: {total_rows}")

print("\nNULL Entries in Any Column:")
if nulls:
    print(f"{len(nulls)} rows with NULLs. Sample:")
    for row in nulls[:5]:
        print(row)
else:
    print("No NULL values found in any column.")

print("\nDuplicate Entries (Ticker + Date):")
if duplicates:
    for row in duplicates:
        print(f"Ticker: {row[0]}, Date: {row[1]}, Count: {row[2]}")
else:
    print("No duplicate (Ticker, Date) pairs found.")

print("\nMissing NYSE Trading Days per Macro Indicator:")
if missing_days:
    for ticker, dates in missing_days.items():
        print(f"{ticker}: {len(dates)} missing (e.g., {dates[:5]}...)")
else:
    print("All macros have full NYSE trading day coverage.")

print("\nMacro Ticker Coverage Check:")
if missing_macros:
    print(f"Missing macro indicators: {missing_macros}")
else:
    print("All 7 expected macro indicators are present.")

if extra_macros:
    print(f"Extra unexpected macro tickers found: {extra_macros}")


Total rows in quant.macro_indicators: 16659

NULL Entries in Any Column:
No NULL values found in any column.

Duplicate Entries (Ticker + Date):
No duplicate (Ticker, Date) pairs found.

Missing NYSE Trading Days per Macro Indicator:
All macros have full NYSE trading day coverage.

Macro Ticker Coverage Check:
All 7 expected macro indicators are present.


# Validate Ticker OHLCV Data

In [30]:
# Get NYSE calendar
nyse = mcal.get_calendar("XNYS")

# Columns to check for NULLs
ohlcv_columns = [
    "ticker", "date", "close", "high", "low", "open", "volume",
    "adjclose", "adjhigh", "adjlow", "adjopen", "adjvolume", "divcash", "splitfactor"
]

with engine.connect() as conn:
    # Total row count
    total_rows = conn.execute(text("SELECT COUNT(*) FROM quant.ticker_ohlcv")).scalar()

    # NULL check
    null_query = f"""
        SELECT *
        FROM quant.ticker_ohlcv
        WHERE {" OR ".join([f'"{col}" IS NULL' for col in ohlcv_columns])};
    """
    nulls = conn.execute(text(null_query)).fetchall()

    # Duplicate check
    duplicates = conn.execute(text("""
        SELECT "ticker", "date", COUNT(*) as count
        FROM quant.ticker_ohlcv
        GROUP BY "ticker", "date"
        HAVING COUNT(*) > 1;
    """)).fetchall()

    # Load ticker metadata
    metadata = conn.execute(text("""
        SELECT "Ticker", "StartDate", "EndDate"
        FROM quant.ticker_metadata;
    """)).fetchall()

    # Load all Ticker-Date pairs
    ohlcv_rows = conn.execute(text("""
        SELECT "ticker", "date"
        FROM quant.ticker_ohlcv;
    """)).fetchall()

# Convert OHLCV data to DataFrame
df_ohlcv = pd.DataFrame(ohlcv_rows, columns=["ticker", "date"])
df_ohlcv["date"] = pd.to_datetime(df_ohlcv["date"]).dt.date

# Check for missing NYSE trading days per ticker
missing_days = {}

for ticker, start, end in metadata:
    nyse_days = nyse.valid_days(start_date=start, end_date=end).date
    actual_dates = set(df_ohlcv[df_ohlcv["ticker"] == ticker]["date"])
    expected_dates = set(nyse_days)
    missing = expected_dates - actual_dates
    if missing:
        missing_days[ticker] = sorted(missing)

# Display Results
print(f"Total rows in quant.ticker_ohlcv: {total_rows}")

print("\nNULL Entries in Any Column:")
if nulls:
    print(f"{len(nulls)} rows with NULLs. Sample:")
    for row in nulls[:5]:
        print(row)
else:
    print("No NULL values found.")

print("\nDuplicate Entries (ticker + date):")
if duplicates:
    for row in duplicates:
        print(f"Ticker: {row[0]}, Date: {row[1]}, Count: {row[2]}")
else:
    print("No duplicate (ticker, date) pairs found.")

print("\nMissing NYSE Trading Days per Ticker:")
if missing_days:
    for ticker, dates in missing_days.items():
        print(f"{ticker}: {len(dates)} missing dates (e.g., {dates[:5]}...)")
else:
    print("All tickers have full NYSE trading day coverage based on metadata ranges.")


Total rows in quant.ticker_ohlcv: 1264709

NULL Entries in Any Column:
No NULL values found.

Duplicate Entries (ticker + date):
No duplicate (ticker, date) pairs found.

Missing NYSE Trading Days per Ticker:
BF.B: 2128 missing dates (e.g., [datetime.date(2017, 1, 3), datetime.date(2017, 1, 4), datetime.date(2017, 1, 5), datetime.date(2017, 1, 6), datetime.date(2017, 1, 9)]...)
BRK.B: 2128 missing dates (e.g., [datetime.date(2017, 1, 3), datetime.date(2017, 1, 4), datetime.date(2017, 1, 5), datetime.date(2017, 1, 6), datetime.date(2017, 1, 9)]...)


In [31]:
with engine.begin() as conn:
    # Update BF.B to BF-B
    conn.execute(text("""
        UPDATE quant.ticker_metadata
        SET "Ticker" = 'BF-B'
        WHERE "Ticker" = 'BF.B';
    """))

    # Update BRK.B to BRK-B
    conn.execute(text("""
        UPDATE quant.ticker_metadata
        SET "Ticker" = 'BRK-B'
        WHERE "Ticker" = 'BRK.B';
    """))

print("Ticker names updated successfully.")

Ticker names updated successfully.


In [32]:
with engine.connect() as conn:
    results = conn.execute(text("""
        SELECT *
        FROM quant.ticker_metadata
        WHERE "Ticker" IN ('BF-B', 'BRK-B');
    """)).fetchall()

for row in results:
    print(row)

('BF-B', '2017-01-01', '2025-06-22', 'XLP')
('BRK-B', '2017-01-01', '2025-06-22', 'XLF')


In [33]:
# Get NYSE calendar
nyse = mcal.get_calendar("XNYS")

# Columns to check for NULLs
ohlcv_columns = [
    "ticker", "date", "close", "high", "low", "open", "volume",
    "adjclose", "adjhigh", "adjlow", "adjopen", "adjvolume", "divcash", "splitfactor"
]

with engine.connect() as conn:
    # Total row count
    total_rows = conn.execute(text("SELECT COUNT(*) FROM quant.ticker_ohlcv")).scalar()

    # NULL check
    null_query = f"""
        SELECT *
        FROM quant.ticker_ohlcv
        WHERE {" OR ".join([f'"{col}" IS NULL' for col in ohlcv_columns])};
    """
    nulls = conn.execute(text(null_query)).fetchall()

    # Duplicate check
    duplicates = conn.execute(text("""
        SELECT "ticker", "date", COUNT(*) as count
        FROM quant.ticker_ohlcv
        GROUP BY "ticker", "date"
        HAVING COUNT(*) > 1;
    """)).fetchall()

    # Load ticker metadata
    metadata = conn.execute(text("""
        SELECT "Ticker", "StartDate", "EndDate"
        FROM quant.ticker_metadata;
    """)).fetchall()

    # Load all Ticker-Date pairs
    ohlcv_rows = conn.execute(text("""
        SELECT "ticker", "date"
        FROM quant.ticker_ohlcv;
    """)).fetchall()

# Convert OHLCV data to DataFrame
df_ohlcv = pd.DataFrame(ohlcv_rows, columns=["ticker", "date"])
df_ohlcv["date"] = pd.to_datetime(df_ohlcv["date"]).dt.date

# Check for missing NYSE trading days per ticker
missing_days = {}

for ticker, start, end in metadata:
    nyse_days = nyse.valid_days(start_date=start, end_date=end).date
    actual_dates = set(df_ohlcv[df_ohlcv["ticker"] == ticker]["date"])
    expected_dates = set(nyse_days)
    missing = expected_dates - actual_dates
    if missing:
        missing_days[ticker] = sorted(missing)

# Display Results
print(f"Total rows in quant.ticker_ohlcv: {total_rows}")

print("\nNULL Entries in Any Column:")
if nulls:
    print(f"{len(nulls)} rows with NULLs. Sample:")
    for row in nulls[:5]:
        print(row)
else:
    print("No NULL values found.")

print("\nDuplicate Entries (ticker + date):")
if duplicates:
    for row in duplicates:
        print(f"Ticker: {row[0]}, Date: {row[1]}, Count: {row[2]}")
else:
    print("No duplicate (ticker, date) pairs found.")

print("\nMissing NYSE Trading Days per Ticker:")
if missing_days:
    for ticker, dates in missing_days.items():
        print(f"{ticker}: {len(dates)} missing dates (e.g., {dates[:5]}...)")
else:
    print("All tickers have full NYSE trading day coverage based on metadata ranges.")


Total rows in quant.ticker_ohlcv: 1264709

NULL Entries in Any Column:
No NULL values found.

Duplicate Entries (ticker + date):
No duplicate (ticker, date) pairs found.

Missing NYSE Trading Days per Ticker:
All tickers have full NYSE trading day coverage based on metadata ranges.
